### We fake the CI-MS as input in SIRIUS and predict formula

In [116]:
import pandas as pd
import os
import re
def adduct_clean(string):
    if string == '[M + H]+':
        return '[M+H]+'
    elif string == '[M]+':
        return '[M]+'
    elif string == '[M - H2 + H]+':
        return '[M-H]+'
    else:
        return 'others'
    
def formula2dict(formula):
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    return res_dct
def formula2dictH(formula):
    lst = re.findall(r'[A-Z][a-z]*|\d+', re.sub('[A-Z][a-z]*(?![\da-z])', r'\g<0>1', formula))
    res_dct = {lst[i]: int(lst[i + 1]) for i in range(0, len(lst), 2)}
    res_dct['H'] = res_dct['H']+1
    return res_dct
def calculate_mass(formula: str):
    #from yuanyue
    
    mol_mass = 0.
    try:
        all_atom_nums = re.findall('([A-Z][a-z]*)([0-9]*)', formula)
        for atom_num in all_atom_nums:
            n = atom_num[1]
            if n == '':
                mol_mass += atom_mass[atom_num[0]]
            else:
                mol_mass += int(n) * atom_mass[atom_num[0]]
    except KeyError as e:
        print("Atom {} is not known".format(e.args[0]))
    except TypeError:
        print('formula is nan')
    return round(mol_mass,6)

In [117]:
atom_mass = {'H': 1.00782503223,
             'C': 12,
             'N': 14.00307400443,
             'O': 15.99491461957,
             'F': 18.99840316273,
             'P': 30.97376199842,
             'S': 31.9720711744,
             'Cl': 34.968852682,
             'Ar': 39.9623831237,
             'K': 38.9637064864,
             'Ca': 39.962590863,
             'Si': 27.976926535,
             'Na':22.98976928
             }

In [118]:
formula = pd.read_csv('./test/filter_all.csv')

In [119]:
formula

,index,name,inchikey,formula,ionization,precursormz
0,0,norvaline,SNDPXSYFESPGGJ-BYPYZUCNSA-N,C8H19N1O2Si1,[M+H]+,190.12618
1,1,norvaline,SNDPXSYFESPGGJ-BYPYZUCNSA-N,C11H27N1O2Si2,[M+H]+,262.16580
2,2,glucosamine 1,MSWZFWKMSRAUBD-QZABAPFNSA-N,C18H45N1O5Si4,[M+H]+,468.24448
3,3,glucosamine 1,MSWZFWKMSRAUBD-QZABAPFNSA-N,C18H45N1O5Si4,[M-H]+,466.22788
4,4,indole-3-propionic acid,GOLXRNDWAUTYKT-UHFFFAOYSA-N,C17H27N1O2Si2,[M+H]+,334.16495
...,...,...,...,...,...,...
628,628,(water loss)4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C11H13N1O1Si1,[M-H]+,202.07201
629,629,4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C14H23N1O2Si2,[M]+,293.12494
630,630,(water loss)4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C11H13N1O1Si1,[M-H]+,202.07201
631,631,4-Hydroxymandelonitrile,HOOOPXDSCKBLFG-UHFFFAOYSA-N,C14H23N1O2Si2,[M]+,293.12494


### Read SIRIUS result

In [120]:
path = './SIRIUS_filter/' #_correct_Mass

In [ ]:
rank_list = []
name = []
adductformula = []
for directory in os.listdir(path):
    
    #directory = '0_annotate_all_1'
    
    if 'filter' in directory:
        
        print(directory)
        index = int(re.findall(r'\d+',directory)[0])
        name.append(index)
        try:
            predict = pd.read_csv(path+directory+'/formula_candidates.tsv', sep='\t')
            key = formula[formula['index'] == int(index)] # get the key row from forumla
            predict['ionization'] = predict['adduct'].apply(adduct_clean)
            predict[predict['ionization'] == key.ionization.item()]
            if key.ionization.item() != '[M]+':
                predict['dict'] = predict['molecularFormula'].apply(formula2dict)
            else:
                predict['dict'] = predict['molecularFormula'].apply(formula2dictH)
                predict = predict[predict['molecularFormula'].str.contains('Si')].reset_index(drop=True)
            keydict = formula2dict(key['formula'].item())
            rank = predict.index[predict['dict'] == keydict].to_list()
            if rank:
                rank_list.append(rank[0])
                adductformula.append(predict['precursorFormula'][rank[0]])
            else:
                rank_list.append('no correct formula')   # in sirius result 
                adductformula.append('no correct formula')
        except FileNotFoundError:
            rank_list.append('nan') # didn't calculate in sirius gui
            adductformula.append('nan')
        except TypeError:
            rank_list.append('no key formula')
            adductformula.append('no key formula')

    

In [123]:
df = pd.DataFrame({'rank':rank_list, 'index':name}) #,'adductformula':adductformula

In [124]:
formula = formula.merge(df,on='index',how='left')

In [125]:
formula.to_csv('rank_filter.csv')

In [126]:
formula['rank'].value_counts()

0                     373
1                      91
no correct formula     71
2                      33
4                      19
3                      17
5                      12
8                       7
6                       6
7                       3
9                       1
Name: rank, dtype: int64